# Sample Source Code

## Pandas HTML + BeautifulSoup

+600 web requests

In [ ]:
!pip install requests-random-user-agent

In [ ]:
import multiprocessing
multiprocessing.cpu_count()

2

In [ ]:

import requests
import requests_random_user_agent
#s = requests.Session()
#print(s.headers['User-Agent'])

# Without a session
resp = requests.get('https://httpbin.org/user-agent')
print(resp.json()['user-agent'])

Mozilla/5.0 (X11; Fedora; Linux x86_64; rv:83.0) Gecko/20100101 Firefox/83.0


In [ ]:
import concurrent.futures
import requests
import requests_random_user_agent
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

def _download_sourceCode(url):

  rq = requests.get(url)
  
  if rq.status_code == 404 or rq.status_code == 403: ## Handle more error codes...
    exit

  main_data = rq.text

  # dataset from table
  df_tempSourceCode = pd.read_html(main_data, index_col=0)[0]

  main_soup = BeautifulSoup(main_data, 'html.parser')
  main_names = main_soup.find_all('tr')[1:245]

  list_urlSourceCode = []

  # Head url for meta_url
  head_Url= 'https://www.programmableweb.com'

  for row in main_names:
      text = row.find_all('td')[0]
      list_urlSourceCode.append( (head_Url + str(text).partition('<a href="')[2].partition('">')[0]))

  df_tempSourceCode['Meta_Url'] = list_urlSourceCode

  return df_tempSourceCode


def download_sourceCode(bulk_urls, num_Splits):
  df_temp = pd.DataFrame()

  # Split urls
  urls_splited  = np.array_split(bulk_urls, num_Splits) # max workers

  tasks = []

  for split in range(len(urls_splited)):
    with concurrent.futures.ThreadPoolExecutor(max_workers = len(urls_splited)) as executor:
      for url in urls_splited[split]:    
        tasks.append(executor.submit(_download_sourceCode, url))
       ## mb call with bulk urls instead of one by one
  # Union
  for result in tasks:
    df_temp = df_temp.append(result.result())

  return df_temp


In [ ]:
sourceCode_urls=[]

for i in range(616): ## web pages?.. 615+1
    main_url = 'https://www.programmableweb.com/category/all/sample-source-code?page=' + str(i) ## parametrizar + comprobar que hay datos
    sourceCode_urls.append(main_url)

In [ ]:
df_sourceCode = pd.DataFrame()
df_sourceCode = download_sourceCode(sourceCode_urls, 15)
df_sourceCode

,Description,Category,Meta_Url
Name of Sample Source Code,,,
Cronj IT Technologies,CronJ is a leading React JS Development Compan...,Software-as-a-Service,https://www.programmableweb.com/sample-source-...
APIclub.in Vehicle Info India PHP Sample Code,This is the sample source code to integrate Ve...,Auto,https://www.programmableweb.com/sample-source-...
Nmap Create scan PHP sample code,PHP sample code for creating Nmap scan with nm...,Security,https://www.programmableweb.com/sample-source-...
Aspose.OMR Cloud NodeJS Examples Source Code,Aspose.OMR Cloud REST API offers SDKs for vari...,Recognition,https://www.programmableweb.com/sample-source-...
Aspose.OMR Cloud Ruby Examples Source Code,Aspose.OMR Cloud REST API offers SDKs for vari...,Recognition,https://www.programmableweb.com/sample-source-...
...,...,...,...
Google Maps Hacking and Bookmarklets,Interesting reference site shows API hacks and...,Mapping,https://www.programmableweb.com/sample-source-...
Flickr Api Explorer,Nice utility that allows you to experiment wit...,Photos,https://www.programmableweb.com/sample-source-...
Yahoo! MapMaker for Microsoft Excel,This Excel template provides an easy way to cr...,Mapping,https://www.programmableweb.com/sample-source-...


In [ ]:
# To see if is any duplicate
df_sourceCode[df_sourceCode.duplicated(keep=False)]

,Description,Category,Meta_Url,Source Code,Repository,Languages
Name of Sample Source Code,,,,,,


## Meta URL Processing

+15k web requests -> split?

TODO> threading


In [ ]:
# Creates new columns
df_sourceCode['Source Code'] = ""
df_sourceCode['Repository'] = ""
df_sourceCode['Languages'] = ""

In [ ]:
def _download_meta_url(df_source):

  for i in range(len(df_source)):

    #print(df_source)
    meta_url = df_source['Meta_Url'][i]
    rq = requests.get(meta_url)

    if rq.status_code == 404 or rq.status_code == 403: ## Handle more error codes...
      continue

    meta_data = rq.text
    meta_soup = BeautifulSoup(meta_data, 'html.parser')

    # Update Description from the meta url
    meta_description = str(meta_soup.find('div', class_='tabs-header_description')).partition('">')[2].partition('</')[0]
    df_source['Description'][i] = meta_description 

    meta_specs = meta_soup.find('div', class_='section specs')

    for lab in meta_specs.select("label"):   

      # Search for Repo
      if (lab.text.lower().find("repository") > -1):
          #print(lab.text + ": " + lab.find_next_sibling().text)
          df_source['Repository'][i] =   lab.find_next_sibling().text

       # Search for Source Code
      if (lab.text.lower().find("link to source code") > -1):
          #print(lab.text + ": " + lab.find_next_sibling().text)
          df_source['Source Code'][i] =   lab.find_next_sibling().text

      # Search for Categories and remplace them
      if (lab.text.lower().find("categories") > -1):
         #print(lab.text + ": " + lab.find_next_sibling().text)
          df_source['Category'][i] =   lab.find_next_sibling().text

      # Search for Languages
      if (lab.text.lower().find("languages") > -1):
          #print(lab.text + ": " + lab.find_next_sibling().text)
          df_source['Languages'][i] =   lab.find_next_sibling().text
    #print(df_source)

  return df_source


def download_meta_url(df_sourceCode, num_Splits):
  # Split dataframe
  dtframe_splited  = np.array_split(df_sourceCode, num_Splits) # max workers
  tasks = []

  for split in range(len(dtframe_splited)):
    with concurrent.futures.ThreadPoolExecutor(max_workers = len(dtframe_splited)) as executor:
      # Download dataframes splited
      tasks.append(executor.submit(_download_meta_url, dtframe_splited[split]))

  df_temp = pd.DataFrame()
  # Union
  for result in tasks:
    df_temp = df_temp.append(result.result())

  return df_temp

In [ ]:
len(dtframe_splited)

In [ ]:
df_sourceCode_metaData = pd.DataFrame()
df_sourceCode_metaData = download_meta_url(df_sourceCode,22)
df_sourceCode_metaData

KeyboardInterrupt: ignored

In [ ]:
df_sourceCode_metaData

""


In [ ]:
# Meta_Url could be used to check for updates on the source website. That uses only +600 web requests instead of +15k
# save a copy of the original dataframe to check for updates based on the meta url or other fields
df_sourceCode_metaData.reset_index(inplace=True)
df_export_sourceCode = df_sourceCode_metaData.copy()

In [ ]:
# Drop the column for the data analysis ( ? ? )
df_sourceCode_metaData.drop('Meta_Url', inplace=True, axis=1)
df_sourceCode_metaData

KeyError: ignored

## Export

In [ ]:
from datetime import datetime
datetime = datetime.now()

## Export the data for analysis 
# To CSV (index True 0,1,2...)
df_export_sourceCode.to_csv(r'/content/DataFrame/sample_source_code_' + datetime.now().strftime('%d_%m_%Y') + '.csv', index = True, header = True)

# To JSON (columns format index True 0,1,2...)
df_export_sourceCode.to_json(r'/content/DataFrame/sample_source_code_' + datetime.now().strftime('%d_%m_%Y') + '.json')


## Export the original + Meta_Url

# To CSV (index True 0,1,2...)
df_sourceCode.to_csv(r'/content/DataFrame/original_sample_source_code_' + datetime.now().strftime('%d_%m_%Y') + '.csv', index = True, header = True)

# To JSON (columns format index True 0,1,2...)
df_sourceCode.to_json(r'/content/DataFrame/original_sample_source_code_' + datetime.now().strftime('%d_%m_%Y') + '.json')